In [1]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import to_categorical

In [2]:
notes = []
counter = 0
for file in glob.glob("midi_songs/*.mid"):
    print('processing song no: ',counter)
    counter = counter + 1
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('+'.join(str(n) for n in element.normalOrder))

processing song no:  0
processing song no:  1
processing song no:  2
processing song no:  3
processing song no:  4
processing song no:  5
processing song no:  6
processing song no:  7
processing song no:  8
processing song no:  9
processing song no:  10
processing song no:  11
processing song no:  12
processing song no:  13
processing song no:  14
processing song no:  15
processing song no:  16
processing song no:  17
processing song no:  18
processing song no:  19
processing song no:  20
processing song no:  21
processing song no:  22
processing song no:  23
processing song no:  24
processing song no:  25
processing song no:  26
processing song no:  27
processing song no:  28
processing song no:  29
processing song no:  30
processing song no:  31
processing song no:  32
processing song no:  33
processing song no:  34
processing song no:  35
processing song no:  36
processing song no:  37
processing song no:  38
processing song no:  39
processing song no:  40
processing song no:  41
pr

In [3]:
n_vocab=len(set(notes))

In [4]:
print('Total notes - ',len(notes))
print('Unique notes - ',len(set(notes)))

Total notes -  45972
Unique notes -  326


In [5]:
print(notes[100:200])

['F3', 'F2', 'F2', 'F2', 'F2', 'F2', '4+9', 'E5', '4+9', 'C5', '4+9', 'A5', '4+9', '5+9', 'F5', '5+9', 'C5', '5+9', 'A5', '5+9', '4+9', 'E5', '4+9', 'C5', '4+9', 'A5', '4+9', 'F5', '5+9', 'C5', '5+9', 'E5', '5+9', 'D5', '5+9', 'E5', '4+9', 'E-5', '4+9', 'B5', '4+9', '4+9', 'A5', '5+9', '5+9', '5+9', '5+9', 'A5', '4+9', '4+9', '4+9', '4+9', '5+9', '5+9', '5+9', '5+9', 'B4', '4+9', 'A4', '4+9', 'E5', '4+9', '4+9', 'E-5', '5+9', '5+9', '5+9', '5+9', 'E-5', '4+9', '4+9', '4+9', '4+9', '5+9', '5+9', '5+9', '5+9', 'E5', '4', 'E-5', 'C6', 'E5', '5', 'E-5', 'B5', 'E5', '6', 'E-5', 'C6', 'A5', '5', 'A4', '4', 'C5', 'E5', 'F5', 'E5', '5', 'C5', 'A4']


In [6]:
sequence_length=100

In [7]:
pitchnames=sorted(set(notes))

In [8]:
ele_to_int=dict((ele,num) for num,ele in enumerate(pitchnames))

In [9]:
network_input=[]
network_output=[]

In [10]:
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    seq_out = notes[i + sequence_length]
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [11]:
n_patterns=len(network_input)
print(n_patterns)

45872


In [12]:
network_input=np.reshape(network_input,(n_patterns,sequence_length,1))
print(network_input.shape)

(45872, 100, 1)


In [13]:
normalised_network_input=network_input/float(n_vocab)

In [14]:
network_output=to_categorical(network_output)

In [15]:
network_output.shape

(45872, 326)

In [16]:
print(network_input.shape)
print(network_output.shape)

(45872, 100, 1)
(45872, 326)


In [17]:
from keras.models import Sequential 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [18]:
model=Sequential()
model.add(LSTM(units=512,input_shape=(normalised_network_input.shape[1],normalised_network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop')

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 512)               2099200   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

In [21]:
# filepath = "model_weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
# checkpoint = ModelCheckpoint(
#     filepath, monitor='loss', 
#     verbose=0,        
#     save_best_only=True,        
#     mode='min'
# )    
# callbacks_list = [checkpoint]     
# hist=model.fit(normalised_network_input, network_output, epochs=100, batch_size=512, callbacks=callbacks_list)

In [23]:
from keras.models import load_model

model=load_model('model_weights/weights-improvement-50-0.1883-bigger.hdf5')

In [24]:
sequence_length=100
network_input=[]
for i in range(len(notes) - sequence_length):
    seq_in=notes[i:i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [25]:
start=np.random.randint(len(network_input) - 1)

int_to_ele=dict((num,ele) for num,ele in enumerate(pitchnames))

pattern=network_input[start]
prediction_output=[]

for note_index in range(200):
    prediction_input=np.reshape(pattern,(1,len(pattern),1))
    prediction_input=prediction_input/float(n_vocab)

    prediction=model.predict(prediction_input,verbose=0)

    idx=np.argmax(prediction)
    result=int_to_ele[idx]
    prediction_output.append(result)

    pattern.append(idx)
    pattern=pattern[1:]

In [26]:
print(prediction_output)

['G2', 'G2', 'F4', 'F#4', 'G4', 'G2', 'B-4', 'G4', 'C3', 'E3', 'C3', 'C3', 'G4', 'E2', 'A4', 'F2', 'B-4', 'F#2', 'B-4', 'B-4', 'G2', 'F3', 'A4', 'B-4', 'C5', 'G2', 'D5', 'G2', 'F5', 'E5', 'G2', 'C3', 'E3', 'C3', 'C3', 'G2', 'F2', 'F2', 'F2', 'F2', 'B-2', 'B-4', 'E5', 'B-2', 'B-2', 'E5', 'B-2', 'F2', 'D5', '9+0', 'E5', 'B-4', 'F2', '9+0', 'F2', '9+0', 'F2', 'B-2', 'B-2', 'B-2', 'B-2', 'G2', 'G2', 'G2', 'G2', 'A2', 'A2', 'D5', 'A2', 'A4', 'F4', 'A2', 'G2', 'G5', 'F5', 'G2', '9+0', 'E5', 'G2', '9+0', 'G2', 'A2', '9+0', 'A2', 'D5', 'A4', 'F4', 'E5', 'F5', '11+2', 'F#5', '11+2', '11+2', 'C#5', 'A4', 'G4', 'E4', 'A5', 'G5', '11+2', 'F#5', '11+2', '11+2', 'C#5', 'A4', 'G4', 'E4', '0+4', '9', 'C5', 'E3', 'B4', 'A3', 'C5', 'C4', '4+9', 'C4', 'C4', 'E4', 'A3', '9', '11+3', '9', 'F#4', 'F#3', 'B4', 'B3', 'E-4', 'F#4', 'E-4', 'F#4', 'B3', 'B4', 'F#3', 'D5', '9', 'B-4', 'F3', 'A4', 'B-3', 'B-4', 'D4', 'F5', 'F4', 'D4', 'D4', 'B-3', 'A2', '1+4', '9', 'E4', 'E3', 'A4', 'A3', 'C#4', 'E4', 'C#4', 'A3',

In [27]:
offset=0
output_notes=[]

for pattern in prediction_output:

    if('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split("+")
        temp_notes=[]
        for current_note in notes_in_chord:
            new_note=note.Note(int(current_note))
            new_note.storedInstrument=instrument.Piano()
            temp_notes.append(new_note)

        new_chord=chord.Chord(temp_notes)
        new_chord.offset=offset
        output_notes.append(new_chord)

    else:
        new_note=note.Note(pattern)
        new_note.offset=offset
        new_note.storedInstrument=instrument.Piano()
        output_notes.append(new_note)

    offset+=0.5

In [28]:
output_notes

[<music21.note.Note G>,
 <music21.note.Note G>,
 <music21.note.Note F>,
 <music21.note.Note F#>,
 <music21.note.Note G>,
 <music21.note.Note G>,
 <music21.note.Note B->,
 <music21.note.Note G>,
 <music21.note.Note C>,
 <music21.note.Note E>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note G>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note F>,
 <music21.note.Note B->,
 <music21.note.Note F#>,
 <music21.note.Note B->,
 <music21.note.Note B->,
 <music21.note.Note G>,
 <music21.note.Note F>,
 <music21.note.Note A>,
 <music21.note.Note B->,
 <music21.note.Note C>,
 <music21.note.Note G>,
 <music21.note.Note D>,
 <music21.note.Note G>,
 <music21.note.Note F>,
 <music21.note.Note E>,
 <music21.note.Note G>,
 <music21.note.Note C>,
 <music21.note.Note E>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note G>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note B->,
 <music2

In [29]:
midi_stream=stream.Stream(output_notes)
midi_stream.write('midi',fp='test_output.mid')

'test_output.mid'

In [30]:
midi_stream.show('midi')